In [ ]:
import logging
import sys
import time

import flimlib
import napari
from napari.qt.threading import thread_worker
import numpy as np
from scipy.spatial import KDTree
import h5py


class ListArray:
    """An array-like object backed by a list of ndarrays."""

    def __init__(self, arrays):
        if not len(arrays):
            raise ValueError # At least for now, don't allow empty

        self._arrays = []
        self._dtype = None
        self._shape = None
        for a in arrays:
            self._arrays.append(np.asarray(a))
            if self._dtype is None:
                self._dtype = self._arrays[0].dtype
            elif self._arrays[-1].dtype != self._dtype:
                raise ValueError
            if self._shape is None:
                self._shape = self._arrays[0].shape
            elif self._arrays[-1].shape != self._shape:
                raise ValueError

    @property
    def ndim(self):
        return 1 + len(self._shape)

    @property
    def dtype(self):
        return self._dtype

    @property
    def shape(self):
        return (len(self._arrays),) + self._shape

    def __getitem__(self, slices):
        if not isinstance(slices, tuple):
            slices = (slices,)
        ndslices = slices[1:]
        s0 = slices[0]
        if isinstance(s0, slice):
            start, stop, step = s0.indices(len(self._arrays))
            dim0 = (stop - start) // step
            shape = (dim0,) + self._shape
            ret = np.empty_like(self._arrays[0], shape=shape)
            j0 = 0
            for i0 in range(start, stop, step):
                ret[j0] = self._arrays[i0][ndslices]
                j0 += 1
            return ret
        else:  # s0 is an integer
            return self._arrays[s0][ndslices]

    def __array__(self):
        return self[:]

In [ ]:
############ Copied from napari/_vispy/cameras.py ############
from vispy.scene.cameras import PanZoomCamera
import numpy as np


class PanZoom1DCamera(PanZoomCamera):
    def __init__(self, axis=1, *args, **kwargs):
        """A camera that can only Pan/Zoom along one axis.
        Useful in a PlotWidget.
        Parameters
        ----------
        axis : int, optional
            The axis to constrain. (This axis will NOT pan or zoom).
                0 => lock x axis
                1 => lock y axis
                by default 1
        """
        self.axis = axis
        super().__init__(*args, **kwargs)

    def zoom(self, factor, center=None):
        if np.isscalar(factor):
            factor = [factor, factor]
        factor[self.axis] = 1
        return super().zoom(factor, center=center)

    def pan(self, pan):
        pan[self.axis] = 0
        self.rect = self.rect + pan


############ Copied from napari/_vispy/vispy_plot.py ############
import numpy as np
from vispy import scene
from vispy.plot import PlotWidget as VispyPlotWidget



# PlotWidget inherits from scene.Widget
class PlotWidget(VispyPlotWidget):
    """Subclass of vispy.plot.PlotWidget.
    Subclassing mostly to override styles (which are not exposed in the main
    class) and to override `_configure_2d` to allow more control over the
    layout.
    """

    # default styles to use for the AxisVisuals
    AXIS_KWARGS = {
        'text_color': 'w',
        'axis_color': 'w',
        'tick_color': 'w',
        'tick_width': 1,
        'tick_font_size': 8,
        'tick_label_margin': 12,
        'axis_label_margin': 50,
        'minor_tick_length': 2,
        'major_tick_length': 5,
        'axis_width': 1,
        'axis_font_size': 9,
    }
    # default styles to use for the title
    TITLE_KWARGS = {'font_size': 16, 'color': 'w'}

    def __init__(
        self,
        margin=10,
        fg_color=None,
        xlabel="",
        ylabel="",
        title="",
        show_yaxis=True,
        show_xaxis=True,
        cbar_length=50,
        axis_kwargs=None,
        title_kwargs=None,
        lock_axis=None,
        **kwargs,
    ):
        self._fg = fg_color
        self.grid = None
        self.camera = None
        self.title = None
        self.title_widget = None
        self.xaxis = None
        self.yaxis = None
        self.xaxis_widget = None
        self.yaxis_widget = None
        self.show_xaxis = show_xaxis
        self.show_yaxis = show_yaxis
        self.xlabel = scene.Label(str(xlabel), color=fg_color or '#ccc')
        self.ylabel = scene.Label(
            str(ylabel), rotation=-90, color=fg_color or '#ccc'
        )
        self.ylabel_widget = None
        self.xlabel_widget = None
        self.padding_left = None
        self.padding_bottom = None
        self.padding_right = None
        self._locked_axis = lock_axis

        self.axis_kwargs = self.AXIS_KWARGS
        if isinstance(axis_kwargs, dict):
            self.axis_kwargs.update(axis_kwargs)
        elif axis_kwargs is not None:
            raise TypeError(
                f'axis_kwargs must be a dict.  got: {type(axis_kwargs)}'
            )
        if fg_color is not None:
            self.axis_kwargs['text_color'] = fg_color
            self.axis_kwargs['axis_color'] = fg_color
            self.axis_kwargs['tick_color'] = fg_color

        self._configured = False
        self.visuals = []

        self.cbar_top = None
        self.cbar_bottom = None
        self.cbar_left = None
        self.cbar_right = None
        self.cbar_length = cbar_length

        super(VispyPlotWidget, self).__init__(**kwargs)
        self.grid = self.add_grid(spacing=0, margin=margin)

        if isinstance(title_kwargs, dict):
            self.TITLE_KWARGS.update(title_kwargs)
        elif title_kwargs is not None:
            raise TypeError(
                f'axis_ktitle_kwargswargs must be a dict.  got: {type(title_kwargs)}'
            )

        self.title = scene.Label(str(title), **self.TITLE_KWARGS)

    def autoscale(self, axes='both'):
        # might be too slow?
        x, y = None, None
        for visual in self.visuals:
            data = None
            if hasattr(visual, '_line'):
                data = np.array(visual._line._pos).T
            elif hasattr(visual, '_markers'):
                data = np.array(visual._markers._data['a_position'])[:, :2].T
            if data is not None and len(data.shape):
                if axes in ('y', 'both'):
                    y = y if y is not None else [np.inf, -np.inf]
                    y[1] = np.maximum(data[1].max(), y[1])
                    y[0] = np.minimum(data[1].min(), y[0])
                if axes in ('x', 'both'):
                    x = x if x is not None else [np.inf, -np.inf]
                    x[1] = np.maximum(data[0].max(), x[1])
                    x[0] = np.minimum(data[0].min(), x[0])
        x = None if np.any(np.isnan(x)) else x
        y = None if np.any(np.isnan(y)) else y
        self.view.camera.set_range(x, y, margin=0.005)

    def _configure_2d(self, fg_color=None):
        if self._configured:
            return

        #         c0        c1      c2      c3      c4      c5         c6
        #     +---------+-------+-------+-------+-------+---------+---------+
        #  r0 |         |                       | title |         |         |
        #     |         +-----------------------+-------+---------+         |
        #  r1 |         |                       | cbar  |         |         |
        #     | ------- +-------+-------+-------+-------+---------+ ------- |
        #  r2 | padding | cbar  | ylabel| yaxis |  view | cbar    | padding |
        #     | ------- +-------+-------+-------+-------+---------+ ------- |
        #  r3 |         |                       | xaxis |         |         |
        #     |         +-----------------------+-------+---------+         |
        #  r4 |         |                       | xlabel|         |         |
        #     |         +-----------------------+-------+---------+         |
        #  r5 |         |                       | cbar  |         |         |
        #     |---------+-----------------------+-------+---------+---------|
        #  r6 |                                 |padding|                   |
        #     +---------+-----------------------+-------+---------+---------+

        # PADDING
        self.padding_right = self.grid.add_widget(None, row=2, col=6)
        self.padding_right.width_min = 1
        self.padding_right.width_max = 5
        self.padding_bottom = self.grid.add_widget(None, row=6, col=4)
        self.padding_bottom.height_min = 1
        self.padding_bottom.height_max = 3

        # TITLE
        self.title_widget = self.grid.add_widget(self.title, row=0, col=4)
        self.title_widget.height_min = self.title_widget.height_max = (
            30 if self.title.text else 5
        )

        # COLORBARS
        self.cbar_top = self.grid.add_widget(None, row=1, col=4)
        self.cbar_top.height_max = 0
        self.cbar_left = self.grid.add_widget(None, row=2, col=1)
        self.cbar_left.width_max = 0
        self.cbar_right = self.grid.add_widget(None, row=2, col=5)
        self.cbar_right.width_max = 0
        self.cbar_bottom = self.grid.add_widget(None, row=5, col=4)
        self.cbar_bottom.height_max = 0

        # Y AXIS
        self.yaxis = scene.AxisWidget(orientation='left', **self.axis_kwargs)
        self.yaxis_widget = self.grid.add_widget(self.yaxis, row=2, col=3)
        if self.show_yaxis:
            self.yaxis_widget.width_max = 30
            self.ylabel_widget = self.grid.add_widget(
                self.ylabel, row=2, col=2
            )
            self.ylabel_widget.width_max = 30 if self.ylabel.text else 1
            self.padding_left = self.grid.add_widget(None, row=2, col=0)
            self.padding_left.width_min = 1
            self.padding_left.width_max = 10
        else:
            self.yaxis.visible = False
            self.yaxis.width_max = 1
            self.padding_left = self.grid.add_widget(
                None, row=2, col=0, col_span=3
            )
            self.padding_left.width_min = 1
            self.padding_left.width_max = 5

        # X AXIS
        self.xaxis = scene.AxisWidget(orientation='bottom', **self.axis_kwargs)
        self.xaxis_widget = self.grid.add_widget(self.xaxis, row=3, col=4)
        self.xaxis_widget.height_max = 20 if self.show_xaxis else 0
        self.xlabel_widget = self.grid.add_widget(self.xlabel, row=4, col=4)
        self.xlabel_widget.height_max = 20 if self.xlabel.text else 0

        # VIEWBOX (this has to go last, see vispy #1748)
        self.view = self.grid.add_view(
            row=2, col=4, border_color=None, bgcolor=None
        )

        self.lock_axis(self._locked_axis)
        self._configured = True
        self.xaxis.link_view(self.view)
        self.yaxis.link_view(self.view)

    def lock_axis(self, axis):
        # work in progress
        if isinstance(axis, str):
            if axis.lower() == 'x':
                axis = 0
            elif axis.lower() == 'y':
                axis = 1
            else:
                raise ValueError("axis must be either 'x' or 'y'")
        self._locked_axis = axis
        if self._locked_axis is not None:
            self.view.camera = PanZoom1DCamera(self._locked_axis)
        else:
            self.view.camera = 'panzoom'
        self.camera = self.view.camera

    def plot(
        self,
        data,
        color=(0.53, 0.56, 0.57, 1.00),
        symbol='o',
        width=1,
        marker_size=8,
        edge_color='gray',
        face_color='gray',
        edge_width=1,
        title=None,
        xlabel=None,
        ylabel=None,
        connect='strip',
    ):
        """Plot a series of data using lines and markers
        Parameters
        ----------
        data : array | two arrays
            Arguments can be passed as ``(Y,)``, ``(X, Y)`` or
            ``np.array((X, Y))``.
        color : instance of Color
            Color of the line.
        symbol : str
            Marker symbol to use.
        width : float
            Line width.
        marker_size : float
            Marker size. If `size == 0` markers will not be shown.
        edge_color : instance of Color
            Color of the marker edge.
        face_color : instance of Color
            Color of the marker face.
        edge_width : float
            Edge width of the marker.
        title : str | None
            The title string to be displayed above the plot
        xlabel : str | None
            The label to display along the bottom axis
        ylabel : str | None
            The label to display along the left axis.
        connect : str | array
            Determines which vertices are connected by lines.
        Returns
        -------
        line : instance of LinePlot
            The line plot.
        See also
        --------
        marker_types, LinePlot
        """
        self._configure_2d()
        line = scene.LinePlot(
            data,
            connect=connect,
            color=color,
            symbol=symbol,
            width=width,
            marker_size=marker_size,
            edge_color=edge_color,
            face_color=face_color,
            edge_width=edge_width,
        )
        self.view.add(line)
        self.visuals.append(line)

        if title is not None:
            self.title.text = title
        if xlabel is not None:
            self.xlabel.text = xlabel
        if ylabel is not None:
            self.ylabel.text = ylabel

        if data is not None:
            self.view.camera.set_range()
        return line

    def scatter(self, data, **kwargs):
        self._configure_2d()
        markers = scene.Markers(pos=data, **kwargs)
        self.view.add(markers)
        self.visuals.append(markers)

        if data is not None:
            self.view.camera.set_range()
        return markers


############ Copied from napari/_vispy/vispy_figure.py ############
from vispy.plot.fig import Fig as VispyFig

class Fig(VispyFig):
    """Subclas of vispy.plot.Fig mostly just to use our internal plot widget.
    """

    def __init__(
        self,
        bgcolor='k',
        size=(800, 600),
        show=True,
        keys=None,
        vsync=True,
        **kwargs,
    ):
        self._plot_widgets = []
        self._grid = None  # initialize before the freeze occurs
        super(VispyFig, self).__init__(
            bgcolor=bgcolor,
            keys=keys,
            show=show,
            size=size,
            vsync=vsync,
            **kwargs,
        )
        self._grid = self.central_widget.add_grid()
        self._grid._default_class = PlotWidget

In [ ]:
period = 0.04
fit_start = 50
fit_end = 210
# selected_pixels has the form ((x1, x2, ...), (y1, y2, ...), ...)
SELECTED_PIXELS = ((100,100),(73,74))
FONT_SIZE = 10
TAU_MAX = 4
CHISQ_MAX = 200
SELECT_RADIUS = 1
MAX_POINTS = 1
PHASOR_SCALE = 1000

# not actually empty. Ideally I could use None as input to napari but it doesn't like it
EMPTY_PLOT = np.zeros((2,1))
EMPTY_IMAGE = np.zeros((1,1))
EMPTY_RGB_IMAGE = np.zeros((1,1,3))
DEFAULT_POINT = np.zeros((1,2))
DEFAULT_PHASOR_POINT = np.asarray([[PHASOR_SCALE,0]])

In [ ]:
hf1 = h5py.File('data_ch2_med_photons.h5', 'r') # change filename
keys= [key for key in hf1.keys()]
photon_count = np.zeros(hf1.get(keys[0]).shape, dtype=np.float32)
for i in range(100):
    photon_count += hf1.get(keys[i])
#selected_photon_count = np.mean(photon_count[((0,),(0,))],axis=0)

In [ ]:
import colorsys

def receive_and_compute(photon_count):
    global period, fit_start, fit_end
    photon_count = np.asarray(photon_count, dtype=np.float32) #does this help?
    phasor = flimlib.GCI_Phasor(period, photon_count, fit_start=fit_start, fit_end=fit_end)
    
    rld = flimlib.GCI_triple_integral_fitting_engine(period, photon_count, fit_start=fit_start, fit_end=fit_end)
    tau = rld.tau
    tau[tau<0] = np.nan
    tau[tau>TAU_MAX] = np.nan # TODO how should the contrast limits be handled?
    tau[rld.chisq > CHISQ_MAX] = np.nan

    intensity = photon_count.sum(axis=-1)
    
    #reshape to work well with mapping / creating the image
    phasor = np.round(np.dstack([(phasor.v * -1 + 1 ) * PHASOR_SCALE, phasor.u * PHASOR_SCALE])).astype(int)


    phasor_quadtree = KDTree(phasor.reshape(-1, 2))

    return 0, 0, intensity, tau, phasor, phasor_quadtree
    

def compute_fits(photon_count):
    global period, fit_start, fit_end
    rld = flimlib.GCI_triple_integral_fitting_engine(period, photon_count, fit_start=fit_start, fit_end=fit_end)
    param_in = [rld.Z, rld.A, rld.tau]
    
    lm = flimlib.GCI_marquardt_fitting_engine(period, photon_count, param_in, fit_start=fit_start, fit_end=fit_end)
    
    return rld, lm

# this would be yeilded
series_no, seqno, intensity, tau, phasor, phasor_quadtree = receive_and_compute(photon_count)

In [ ]:
from vispy.scene.visuals import Text
import time

fit_start = 50
fit_end = 210

def autoscale_viewer(viewer, shape):
    state = {'rect': ((0, 0), shape)}
    viewer.window.qt_viewer.view.camera.set_state(state)

class CurveFittingPlot():
    #TODO add transform into log
    def __init__(self, viewer):
        self.fig = Fig()
        # add a docked figure
        self.dock_widget = viewer.window.add_dock_widget
        # get a handle to the plotWidget
        self.ax = self.fig[0, 0]
        self.lm_curve = self.ax.plot(None, color='g', marker_size=0, width=2)
        self.rld_curve = self.ax.plot(None, color='r', marker_size=0, width=2)
        self.data_scatter = self.ax.scatter(None, size=1, edge_width=0, face_color='m')
        self.fit_start_line = self.ax.plot(None, color='b', marker_size=0, width=2)
        self.fit_end_line = self.ax.plot(None, color='b', marker_size=0, width=2)
        self.rld_info = Text(None, parent=self.ax.view, color='r', anchor_x='right', font_size = FONT_SIZE)
        self.lm_info = Text(None, parent=self.ax.view, color='g', anchor_x='right', font_size = FONT_SIZE)
    
    def update_with_selection(self, selection):
        rld_selected, lm_selected = compute_fits(selection)
        lm_time = np.linspace(0,(lm_selected.fitted.size-1)*period,lm_selected.fitted.size,dtype=np.float32)
        rld_time = lm_time[0:rld_selected.fitted.size]
        self.lm_curve.set_data((lm_time, lm_selected.fitted))
        self.rld_curve.set_data((rld_time, rld_selected.fitted))
        self.data_scatter.set_data(np.array((lm_time, selection)).T, size=3, edge_width=0, face_color='m')
        
        self.rld_info.pos = self.ax.view.size[0], self.rld_info.font_size
        self.rld_info.text = 'RLD | chisq = ' + "{:.2e}".format(float(rld_selected.chisq)) + ', tau = ' + "{:.2e}".format(float(rld_selected.tau))
        self.lm_info.pos = self.ax.view.size[0], self.rld_info.font_size*3
        self.lm_info.text = 'LMA | chisq = ' + "{:.2e}".format(float(lm_selected.chisq)) + ', tau = ' + "{:.2e}".format(float(lm_selected.param[2]))
        
        # autoscale based on data (ignore start/end lines)
        self.fit_start_line.set_data(np.zeros((2,1)))
        self.fit_end_line.set_data(np.zeros((2,1)))
        self.ax.autoscale()
        self.fit_start_line.set_data(([fit_start * period, fit_start * period], self.ax.camera._ylim))
        self.fit_end_line.set_data(([fit_end * period, fit_end * period], self.ax.camera._ylim))

        #TODO figure out where this gets called. after moving/adding a point and also modifying fitstart/fit end

class LifetimeSelectionMetadata():
    def __init__(self, selection, co_selection, decay_plot, photon_count, phasor):
        self.selection = selection
        self.co_selection = co_selection
        self.decay_plot = decay_plot
        self.photon_count = photon_count
        self.phasor = phasor
    def update_co_selection(self):
        points = get_bounded_points(self.selection, self.phasor.shape[:2])
        
        if(len(points) > 0):
            points_indexer = tuple(np.asarray(points).T)
            set_points(self.co_selection, phasor[points_indexer])
            self.update_decay_plot(np.mean(self.photon_count[points_indexer],axis=0))
        else:
            no_data = np.empty(self.photon_count.shape[-1])
            no_data.fill(np.nan)
            self.update_decay_plot(no_data)
            set_points(self.co_selection, None)

    def update_decay_plot(self, selection):
        self.decay_plot.update_with_selection(selection)

# copied code from above with different coselection updating
class PhasorSelectionMetadata():
    def __init__(self, selection, co_selection, decay_plot, photon_count, phasor):
        self.selection = selection
        self.co_selection = co_selection
        self.decay_plot = decay_plot
        self.photon_count = photon_count
        self.phasor = phasor

    def update_co_selection(self):
        selection_radius = int(self.selection.current_size) - 1 # TODO: Use correct selection radius
        selection_center = self.selection.data[0]

        height, width, _ = phasor.shape
        maxpoints = width * height
        distances, indices = phasor_quadtree.query(selection_center, maxpoints, distance_upper_bound=selection_radius)
        n_indices = np.searchsorted(distances, np.inf)
        indices = indices[0:n_indices]

        x, y = indices % width, indices // height 
        set_points(self.co_selection, np.column_stack((y, x)))

        histograms = self.photon_count[y, x]
        print("histograms", histograms.shape)
        if not len(histograms):
            histograms = np.zeros(self.photon_count.shape[-1]) + np.nan
        print("histograms", histograms.shape)
        histogram = np.mean(histograms, axis=0)
        print("histogram", histogram.shape)
        self.update_decay_plot(histogram)

    def update_decay_plot(self, selection):
        self.decay_plot.update_with_selection(selection)

def get_points(layer):
        point_size = int(layer.current_size) - 1 #THIS IS NOT CORRECT (I have no idea what napari considers as point size)
        points = []      
        for point in layer.data.astype(int):
            for r in range(point[0] - point_size, point[0] + point_size + 1):
                for c in range(point[1] - point_size, point[1] + point_size + 1):
                    points += [[r, c]]
        return points

def get_bounded_points(layer, image_shape):
        points = []
        point_size = int(layer.current_size) - 1 #THIS IS NOT CORRECT (I have no idea what napari considers as point size)
        for point in layer.data.astype(int):
            rmax = image_shape[0] - 1
            cmax = image_shape[1] - 1
            for r in range(min(rmax, max(0, point[0] - point_size)), min(rmax, max(0, point[0] + point_size + 1))):
                for c in range(min(cmax, max(0, point[1] - point_size)), min(cmax, max(0, point[1] + point_size + 1))):
                    points += [[r, c]]
        return points

def update_lifetime_image(tau, intensity):
    global lifetime_image
    intensity *= 1.0/intensity.max()
    tau *= 1.0/np.nanmax(tau)
    intensity_scaled_tau = np.zeros([*tau.shape,3], dtype=float)

    # temporary colormap (mark suggested a dict map)
    for r in range(tau.shape[0]):
        for c in range(tau.shape[1]):
            if not np.isnan(tau[r][c]):
                intensity_scaled_tau[r][c] = colorsys.hsv_to_rgb(tau[r][c], 1.0, intensity[r][c])

    lifetime_image.data = intensity_scaled_tau

def update_phasor_image(phasor, intensity):
    global phasor_image
    set_points(phasor_image, phasor.reshape(-1,2), intensity=intensity.ravel() * .1)
    phasor_image.editable = False

def set_points(points_layer, points, intensity=None):
    points_layer.data = points if len(points) else None
    
    if intensity is not None:
        color = np.lib.stride_tricks.as_strided([1.0], shape=(points.shape[0],), strides=(0,))
        points_layer.face_color = np.array([color,color,color,intensity]).T
    points_layer.selected_data = {}

def select_points_drag(layer, event):
    try:
        layer.metadata['selection'].update_co_selection()
        yield
        while event.type == 'mouse_move':
            layer.metadata['selection'].update_co_selection()
            yield
    except Exception as e:
        print("select_points_drag")
        print(event.type)
        print(e)

def handle_new_point(event):
    event_layer = event._sources[0]
    try:
        # make sure to check if each of these operations has already been done since
        # changing the data triggers this event which may cause infinite recursion
        if event_layer.data.shape[0] > 0 and event_layer.data.dtype != int:
            event_layer.data = np.round(event_layer.data).astype(int)
        if event_layer.data.shape[0] > 1 and event_layer.editable:
            event_layer.data = event_layer.data[-1:]
        if event_layer.data.shape[0] > 0:
            if('selection' in event_layer.metadata):
                event_layer.metadata['selection'].update_co_selection()
    except Exception as e:
        print("handle_new_point")
        print(e)

def create_lifetime_select_layer(viewer, co_viewer, decay_plot, photon_count, phasor):
    selection = viewer.add_points(DEFAULT_POINT, name="Selection", symbol="square", face_color="#ffffff7f", edge_width=0)
    co_selection = co_viewer.add_points(None, name="Correlation", size=1, face_color='red', edge_width=0)
    co_selection.editable = False
    selection.metadata = {'selection': LifetimeSelectionMetadata(selection, co_selection, decay_plot, photon_count, phasor)}
    selection.mouse_drag_callbacks.append(select_points_drag)
    selection.events.data.connect(handle_new_point)
    selection.mode = 'select'
    return selection

def create_phasor_select_layer(viewer, co_viewer, decay_plot, photon_count, phasor):
    selection = viewer.add_points(DEFAULT_POINT, name="Selection", symbol="square", face_color="#ffffff7f", edge_width=0)
    co_selection = co_viewer.add_points(None, name="Correlation", size=1, face_color='red', edge_width=0)
    co_selection.editable = False
    selection.metadata = {'selection': PhasorSelectionMetadata(selection, co_selection, decay_plot, photon_count, phasor)}
    selection.mouse_drag_callbacks.append(select_points_drag)
    selection.events.data.connect(handle_new_point)
    selection.mode = 'select'
    return selection

In [ ]:
try:
    if lifetime_viewer:
        lifetime_viewer.close()
except:
    print("viewer already closed or never opened")
    
lifetime_viewer = napari.Viewer(title="Lifetime Viewer")
lifetime_image = lifetime_viewer.add_image(EMPTY_RGB_IMAGE, rgb=True, name="Lifetime")
update_lifetime_image(tau, intensity)
autoscale_viewer(lifetime_viewer, tau.shape)

try:
    if phasor_viewer:
        phasor_viewer.close()
except:
    print("viewer already closed or never opened")

phasor_viewer = napari.Viewer(title="Phasor Viewer")
phasor_image = phasor_viewer.add_points(None, name="Phasor", edge_width=0, size = 3)
phasor_image.editable = False
update_phasor_image(phasor, intensity)
autoscale_viewer(phasor_viewer, (PHASOR_SCALE, PHASOR_SCALE))

#add the phasor circle
phasor_circle = np.asarray([[PHASOR_SCALE, 0.5 * PHASOR_SCALE],[0.5 * PHASOR_SCALE,0.5 * PHASOR_SCALE]])
x_axis_line = np.asarray([[PHASOR_SCALE,0],[PHASOR_SCALE,PHASOR_SCALE]])
phasor_shapes_layer = phasor_viewer.add_shapes([phasor_circle, x_axis_line], shape_type=['ellipse','line'], face_color='',)
phasor_shapes_layer.editable = False

curve_fitting_plot = CurveFittingPlot(lifetime_viewer)
lifetime_select = create_lifetime_select_layer(lifetime_viewer, phasor_viewer, curve_fitting_plot, photon_count, phasor)
phasor_select = create_phasor_select_layer(phasor_viewer, lifetime_viewer, curve_fitting_plot, photon_count, phasor)

In [ ]:
def update_image_layers():
    global current_series_frames, phasor_points, phasor, intensity, tau
    series_no, seqno, intensity, tau, phasor = receive_and_compute(photon_count)
    
    update_lifetime_image(tau, intensity)
    update_phasor_image(phasor, intensity)

from magicgui import magicgui

@magicgui(auto_call=True, 
    start={"label": "Fit Start", "max": photon_count.shape[-1] * period}, 
    end={"label": "Fit End", "max": photon_count.shape[-1] * period}
    )
def options_widget(
    start : float = fit_start * period,
    end : float = fit_end * period,
):
    try:
        global fit_start, fit_end
        fit_start = int(start/period)
        fit_end = int(end/period)
        #FOR EACH IN THE MAPPING
        
        update_image_layers()

        for layer in lifetime_viewer.layers:
            if 'selection' in layer.metadata:
                layer.metadata['selection'].update_co_selection()
        for layer in phasor_viewer.layers:
            if 'selection' in layer.metadata:
                layer.metadata['selection'].update_co_selection()
    except Exception as e:
        print(e)
    
lifetime_viewer.window.add_dock_widget(options_widget)